In [ ]:
#大气校正后的地面反射率，通过SBAF校正（波段一致性差异）后，模拟目标传感器接收到的天顶反射率

In [ ]:
#处理光谱响应函数
import pandas as pd
import numpy as np
from scipy import interpolate
RSR_dir=r'E:/code/HY_reprocessing/parameterFile/RSR\HY1B_OCT_RSR.txt'  # 第一列为波长
dataSpectrum=pd.read_table(RSR_dir,index_col=None,delim_whitespace=True)

wavelength=dataSpectrum.iloc[:,0]/1000.0                             #微米w
wavelength_new=np.arange(wavelength.min(),wavelength.max(),0.0025)    #0.025微米的光谱间隔
header=['wavelength','b1','b2','b3','b4','b5','b6','b7','b8']
rsr_6s=pd.DataFrame()
rsr_6s["wavelength"]=wavelength_new
#对光谱响应函数进行插值
for i in range(1,dataSpectrum.shape[1]):                             #每个波段插值,一次算一个波段
    rrs=(dataSpectrum.iloc[:,i].values)/100.0
    f=interpolate.interp1d(wavelength,rrs,kind="linear")
    rrs_new=f(wavelength_new)
    k=str(header[i])
    rsr_6s[k]=rrs_new                                                    #插值结果存入dataframe
print(rsr_6s)
#return df, wavelength.min(), wavelength.max()                        #返回一个经过插值的光谱响应函数，间隔为0.025微米

In [ ]:
def six_s(s_z,s_a,v_z,v_a,band,ref):
    print(s_z,s_a,v_z,v_a,band,ref)
    if ref < 0.0:
        out=np.nan
    else:
        s = SixS()
        #大气剖面和气溶胶
        s.atmos_profile = AtmosProfile.PredefinedType(AtmosProfile.MidlatitudeSummer)
        s.aero_profile=AeroProfile.PredefinedType(AeroProfile.Maritime)

        # 水汽和臭氧
        #s.atmosprofile = AtmosProfile.UserWaterAndOzone(1.84604, 1.9729 ) # 水汽和臭氧的单位197.29

        #地表反射率
        s.ground_reflectance=GroundReflectance.HomogeneousLambertian(ref)#GroundReflectance.ClearWater)#ClearWater,Sand,Lakewater
        #观测几何
        s.geometry.solar_z=s_z#31.656956
        s.geometry.solar_a=s_a#135.71416
        s.geometry.view_z=v_z#22.372787
        s.geometry.view_a=v_a#100.96758
        s.geometry.day=2
        s.geometry.month=10
        #卫星和目标高度
        s.altitudes.set_sensor_satellite_level()#set_sensor_custom_altitude(500)
        s.altitudes.set_target_custom_altitude(0)
        #s.wavelength=Wavelength(PredefinedWavelengths.ACCURATE_MODIS_TERRA_8)
        #s.wavelength=Wavelength(rsr_6s.min()[0],rsr_6s.max()[0],(list(rsr_6s.iloc[:,band])).append(0))
        s.wavelength=Wavelength(0.35,1.1,list(rsr_6s.iloc[:,band]).append(0))
        s.run()
        out=s.outputs.apparent_radiance
    yield out

In [ ]:
#主函数，根据seadas大气校正结果和MOD03的观测几何，作为输入，用6s模拟TOA，并写出到H5文件
import h5py,glob,os
from pyhdf.SD import SD,SDC
import numpy as np
from Py6S import *
if __name__=='__main__':
    files=glob.glob(r'G:\vm\share\problem\*L2_LAC_OC')  

    for i,file in enumerate(files):
        MOD03=SD(r'G:\vm\share\problem\MOD03.A2008094.0225.061.2017255024942.hdf',SDC.READ|SDC.WRITE)
        #空间范围
        up=1645
        down=1698
        left=363
        right=638
        sz=MOD03.select('SolarZenith')[up:down,left:right]*0.01
        sa=MOD03.select('SolarAzimuth')[up:down,left:right]*0.01
        vz=MOD03.select('SensorZenith')[up:down,left:right]*0.01
        va=MOD03.select('SensorAzimuth')[up:down,left:right]*0.01
        
        nl=down-up
        ns=right-left
        SBAF=[1.00187, 1.00124,0.99993,1.00224,0.978365,0.99609,1.332186 ,1.0864677]
        geophysical_para=['Rrs_412','Rrs_443','Rrs_488','Rrs_531','Rrs_555','Rrs_678','Rrs_748','Rrs_869']
        TOA_para=['L_412','L_443','L_488','L_531','L_555','L_678','L_748','L_869']
        
        TOAfile=file[0:-4]+'_TOAradiance.H5'
        if os.access(TOAfile,os.R_OK)==True:
            os.remove(TOAfile) 
            
        
        h5ID=h5py.File(file,'r')
        h5_new=h5py.File(TOAfile,'w')
        geophysical=h5_new.create_group('Geophysical Data')
        for k,ID in enumerate(geophysical_para): 
            Rrs=h5ID['geophysical_data/'+ID][up:down,left:right]
            offset=h5ID['geophysical_data/'+ID].attrs['add_offset']
            scale=h5ID['geophysical_data/'+ID].attrs['scale_factor']
            #Rrs=Rrs*1.0
            #Rrs[Rrs==-32767.0]=np.nan
            Rrs=Rrs*scale+0.05
            Rrs=Rrs*SBAF[k]
            bandn=np.full((nl,ns),k+1)
            radiance=np.array([*map(lambda s_z,s_a,v_z,v_a,band_x,ref:six_s(s_z,s_a,v_z,v_a,band_x,ref).__next__(),sz.flatten(),\
                      sa.flatten(),vz.flatten(),va.flatten(),bandn.flatten(),Rrs.flatten())])
            radiance=radiance.reshape(nl,ns)
            band=geophysical.create_dataset(TOA_para[k],(nl,ns), dtype='f',data = radiance)
        h5_new.close()
        h5ID.close()
            
        

In [ ]:
#py6s模拟，写出到tif
import numpy as np
import numba as nb
from Py6S import *
#模拟函数
def six_s(s_z,s_a,v_z,v_a,band,ref):
    if ref 
    s = SixS()
    #大气剖面和气溶胶
    s.atmos_profile = AtmosProfile.PredefinedType(AtmosProfile.AtmosProfile)
    s.atmos_profile = AtmosProfile.FromLatitudeAndDate(latitude,date)
    s.aero_profile=AeroProfile.PredefinedType(AeroProfile.Maritime)

    # 水汽和臭氧
    #s.atmosprofile = AtmosProfile.UserWaterAndOzone(1.84604, 1.9729 ) # 水汽和臭氧的单位197.29

    #地表反射率
    s.ground_reflectance=GroundReflectance.HomogeneousLambertian(0.05)#GroundReflectance.ClearWater)#ClearWater,Sand,Lakewater
    #观测几何
    s.geometry.solar_z=s_z#31.656956
    s.geometry.solar_a=s_a#135.71416
    s.geometry.view_z=v_z#22.372787
    s.geometry.view_a=v_a#100.96758
    s.geometry.day=8
    s.geometry.month=10
    #卫星和目标高度
    s.altitudes.set_sensor_satellite_level()#set_sensor_custom_altitude(500)
    s.altitudes.set_target_custom_altitude(0)
    #s.wavelength=Wavelength(PredefinedWavelengths.ACCURATE_MODIS_TERRA_8)
    s.wavelength=Wavelength(rsr_6s.min(),rsr_6s.max(),list(rsr_6s.iloc[:,band]))
    s.run()
    yield s.outputs.apparent_radiance

def write_img(filename,im_proj,im_geotrans,im_data):
    #gdal数据类型包括
    #gdal.GDT_Byte, 
    #gdal .GDT_UInt16, gdal.GDT_Int16, gdal.GDT_UInt32, gdal.GDT_Int32,
    #gdal.GDT_Float32, gdal.GDT_Float64

    #判断栅格数据的数据类型
    if 'int8' in im_data.dtype.name:
        datatype = gdal.GDT_Byte
    elif 'int16' in im_data.dtype.name:
        datatype = gdal.GDT_UInt16
    else:
        datatype = gdal.GDT_Float32

    #判读数组维数
    if len(im_data.shape) == 3:
        im_bands, im_height, im_width = im_data.shape
    else:
        im_bands, (im_height, im_width) = 1,im_data.shape 

    #创建文件
    driver = gdal.GetDriverByName("GTiff")            #数据类型必须有，因为要计算需要多大内存空间
    dataset = driver.Create(filename, im_width, im_height, im_bands, datatype)

    dataset.SetGeoTransform(im_geotrans)              #写入仿射变换参数
    dataset.SetProjection(im_proj)                    #写入投影

    if im_bands == 1:
        dataset.GetRasterBand(1).WriteArray(im_data)  #写入数组数据
    else:
        for i in range(im_bands):
            dataset.GetRasterBand(i+1).WriteArray(im_data[i])

    del dataset
if __name__ == "__main__":
    os.chdir(r'D:\Python_Practice')                        #切换路径到待处理图像所在文件夹
    src_ds=gdal.Open(filename)
    nb=src_ds.RasterCount
    im_geotrans = dataset.GetGeoTransform()               #仿射矩阵
    im_proj = dataset.GetProjection()  
    #读取几何
    sz=src_ds.GetRasterBand().ReadAsArray()
    sa=src_ds.GetRasterBand().ReadAsArray()
    vz=src_ds.GetRasterBand().ReadAsArray()
    va=src_ds.GetRasterBand().ReadAsArray()
    
    @nb.jit(nopython=True)
    for band in range(nb):
        band+=1
        srcband=src_ds.GetRasterBand(band)
        im_data=srcband.ReadAsArray()
        bandn=np.full((srcband.RasterYSize,srcband.RasterXSize),band)
        radiance=np.array([*map(lambda s_z,s_a,v_z,v_a,band_x,ref:sz.flatten(),six_s(s_z,s_a,v_z,v_a,band_x,ref),\
            sa.flatten(),vz.flatten(),va.flatten(),bandn.flatten(),im_data.flatten())])
        out_data=radiance.reshape(srcband.RasterYSize,srcband.RasterXSize)
        outfile=filename[:-4]+'_band'+str(band)+'.tif'
        write_img(outfile,im_proj,im_geotrans,out_data)